In [1]:
from sklearn import tree
import pandas as pd
import os

  ## Importing and cleaning the training dataset

In [2]:
data = pd.read_csv('../../Resources/bio_vars_frame.csv')
data.head()

,Unnamed: 0,bio_1,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,bio_9,...,bio_11,bio_12,bio_13,bio_14,bio_15,bio_16,bio_17,bio_18,bio_19,raster_frame
0,1,96.0,105.0,40.0,5390.0,250.0,-7.0,257.0,35.0,165.0,...,30.0,2184.0,350.0,25.0,61.0,1008.0,144.0,147.0,951.0,0
1,2,94.0,105.0,40.0,5385.0,249.0,-9.0,258.0,34.0,163.0,...,28.0,2196.0,354.0,24.0,62.0,1019.0,142.0,146.0,961.0,0
2,3,96.0,105.0,40.0,5407.0,251.0,-7.0,258.0,35.0,165.0,...,30.0,2162.0,348.0,24.0,62.0,1002.0,140.0,144.0,945.0,0
3,4,97.0,105.0,40.0,5437.0,251.0,-7.0,258.0,35.0,166.0,...,30.0,2144.0,345.0,24.0,62.0,993.0,139.0,143.0,937.0,0
4,5,93.0,105.0,40.0,5438.0,248.0,-11.0,259.0,32.0,163.0,...,27.0,2188.0,357.0,24.0,63.0,1023.0,138.0,143.0,966.0,0


In [3]:
data.drop(['Unnamed: 0'], axis = 1, inplace = True)
data.dropna(axis = 0, how = 'any', inplace = True)
data.rename(columns={'bio_1': 'Annual Mean Temperature', 
                     'bio_2': 'Mean Diurnal Range (Mean of monthly (max temp - min temp))',
                     'bio_3': 'Isothermality',
                     'bio_4': 'Temperature Seasonality',
                     'bio_5': 'Max Temperature of Warmest Month',
                     'bio_6': 'Min Temperature of Coldest Month',
                     'bio_7': 'Temperature Annual Range',
                     'bio_8': 'Mean Temperature of Wettest Quarter',
                     'bio_9': 'Mean Temperature of Driest Quarter',
                     'bio_10': 'Mean Temperature of Warmest Quarter',
                     'bio_11': 'Mean Temperature of Coldest Quarter',
                     'bio_12': 'Annual Precipitation',
                     'bio_13': 'Precipitation of Wettest Month',
                     'bio_14': 'Precipitation of Driest Month',
                     'bio_15': 'Precipitation Seasonality (Coefficient of Variation)',
                     'bio_16': 'Precipitation of Wettest Quarter',
                     'bio_17': 'Precipitation of Driest Quarter',
                     'bio_18': 'Precipitation of Warmest Quarter',
                     'bio_19': 'Precipitation of Coldest Quarter',
                    }, inplace=True)

data = data[['raster_frame','Precipitation of Driest Month', 'Temperature Seasonality', 'Precipitation of Driest Quarter',
             'Annual Precipitation', 'Precipitation of Warmest Quarter']]

data.head()

,raster_frame,Precipitation of Driest Month,Temperature Seasonality,Precipitation of Driest Quarter,Annual Precipitation,Precipitation of Warmest Quarter
0,0,25.0,5390.0,144.0,2184.0,147.0
1,0,24.0,5385.0,142.0,2196.0,146.0
2,0,24.0,5407.0,140.0,2162.0,144.0
3,0,24.0,5437.0,139.0,2144.0,143.0
4,0,24.0,5438.0,138.0,2188.0,143.0


## Preprocessing the data for model training

In [4]:
target = data['raster_frame']
target_names = ["negative", "positive"]

In [5]:
X = data.drop(["raster_frame"], axis=1)
feature_names = X.columns
X.head()

,Precipitation of Driest Month,Temperature Seasonality,Precipitation of Driest Quarter,Annual Precipitation,Precipitation of Warmest Quarter
0,25.0,5390.0,144.0,2184.0,147.0
1,24.0,5385.0,142.0,2196.0,146.0
2,24.0,5407.0,140.0,2162.0,144.0
3,24.0,5437.0,139.0,2144.0,143.0
4,24.0,5438.0,138.0,2188.0,143.0


In [9]:
# Splitting up the data for training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, target, random_state=42)

In [10]:
# Testing accuracy on training and testing data
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.933613317695993

## Making a prediction on test data set

In [11]:
# Making prediction based on test data
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
prediction = rf.predict_proba(X_test)
print(prediction)

[[1.   0.  ]
 [0.62 0.38]
 [1.   0.  ]
 ...
 [1.   0.  ]
 [1.   0.  ]
 [1.   0.  ]]


In [12]:
# Looking at important of each feature
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.4110402607262112, 'Temperature Seasonality'),
 (0.2750618175301082, 'Annual Precipitation'),
 (0.14188196013138799, 'Precipitation of Warmest Quarter'),
 (0.10317890791826062, 'Precipitation of Driest Quarter'),
 (0.06883705369403197, 'Precipitation of Driest Month')]

## Importing and cleaning new data set

In [15]:
mild_2050 = pd.read_csv('../../Resources/2050_mild.csv')

In [16]:
mild_2050.drop(['Unnamed: 0'], axis = 1, inplace = True)
mild_2050.dropna(axis = 0, how = 'any', inplace = True)
mild_2050.rename(columns={'bc45bi501': 'Annual Mean Temperature', 
                     'bc45bi5010': 'Mean Temperature of Warmest Quarter',
                     'bc45bi5011': 'Mean Temperature of Coldest Quarter',
                     'bc45bi5012': 'Annual Precipitation',
                     'bc45bi5013': 'Precipitation of Wettest Month',
                     'bc45bi5014': 'Precipitation of Driest Month',
                     'bc45bi5015': 'Precipitation Seasonality (Coefficient of Variation)',
                     'bc45bi5016': 'Precipitation of Wettest Quarter',
                     'bc45bi5017': 'Precipitation of Driest Quarter',
                     'bc45bi5018': 'Precipitation of Warmest Quarter',
                     'bc45bi5019': 'Precipitation of Coldest Quarter',
                     'bc45bi502': 'Mean Diurnal Range (Mean of monthly (max temp - min temp))',
                     'bc45bi503': 'Isothermality',
                     'bc45bi504': 'Temperature Seasonality',
                     'bc45bi505': 'Max Temperature of Warmest Month',
                     'bc45bi506': 'Min Temperature of Coldest Month',
                     'bc45bi507': 'Temperature Annual Range',
                     'bc45bi508': 'Mean Temperature of Wettest Quarter',
                     'bc45bi509': 'Mean Temperature of Driest Quarter',
                    }, inplace=True)

mild_2050.head()

,x,y,Annual Mean Temperature,Mean Temperature of Warmest Quarter,Mean Temperature of Coldest Quarter,Annual Precipitation,Precipitation of Wettest Month,Precipitation of Driest Month,Precipitation Seasonality (Coefficient of Variation),Precipitation of Wettest Quarter,...,Precipitation of Warmest Quarter,Precipitation of Coldest Quarter,Mean Diurnal Range (Mean of monthly (max temp - min temp)),Isothermality,Temperature Seasonality,Max Temperature of Warmest Month,Min Temperature of Coldest Month,Temperature Annual Range,Mean Temperature of Wettest Quarter,Mean Temperature of Driest Quarter
0,-124.779167,49.3375,102,172,45,1842,327,31,66,893,...,133,793,87,37,4993,249,14,235,52,168
1,-124.770833,49.3375,99,168,43,1860,329,31,66,899,...,136,798,85,36,4915,243,12,231,50,164
2,-124.762500,49.3375,97,166,42,1866,331,32,66,901,...,137,801,85,36,4909,241,10,231,48,162
3,-124.754167,49.3375,105,174,47,1810,321,30,67,879,...,130,779,87,36,5015,252,16,236,54,171
4,-124.745833,49.3375,112,183,51,1764,314,27,68,863,...,122,765,89,36,5143,264,20,244,58,180


In [17]:
x_new = mild_2050[['Precipitation of Driest Month', 'Temperature Seasonality', 'Precipitation of Driest Quarter',
             'Annual Precipitation', 'Precipitation of Warmest Quarter']]
x_new.head()

,Precipitation of Driest Month,Temperature Seasonality,Precipitation of Driest Quarter,Annual Precipitation,Precipitation of Warmest Quarter
0,31,4993,122,1842,133
1,31,4915,125,1860,136
2,32,4909,127,1866,137
3,30,5015,118,1810,130
4,27,5143,108,1764,122


## Making a prediction on our new data

In [18]:
mild2050_prediction = rf.predict_proba(x_new)

## Creating final dataframe and exporting

In [20]:
mild2050_final = harsh_2050[['x', 'y']]
mild2050_final["Prediction"] = mild2050_prediction[:,1]
mild2050_final.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,x,y,Prediction
0,-124.779167,49.3375,0.00
1,-124.770833,49.3375,0.00
2,-124.762500,49.3375,0.00
3,-124.754167,49.3375,0.00
4,-124.745833,49.3375,0.02


In [21]:
mild2050_final.to_csv('2050_moderate_final.csv')